# Planilhas SFMSP

**Problema:** Estrair os dados de arquivos PDF. O arquivo é dividido em duas partes:
- Cabeçalho: Dados pessoal do servidor
- Tabela: Dados de insalubridade mensal divido em duas partes:
  - Valor Recebido: INSAL. %, VALOR, 6ª PTE, FÉRIAS, Q. DIAS  
  - Valor Devido: V. INSAL., 6ªPTE, FÉRIAS, A RECEB.

## Bibliotecas

In [376]:
import os
import re
import tabula
import fitz as fz
from random import random

## Código

### Diretórios

In [377]:
BASE_DIR = os.getcwd()
DATA_DIR = f'{BASE_DIR}\\data'
DATA_FILES = os.listdir(DATA_DIR)
total_itens = len(DATA_FILES)

### Ler conteúdo do arquivo

In [378]:
def ler_dados(DATA_FILE):
        with fz.open(f'{DATA_DIR}\\{DATA_FILE}') as file:
                context = ''
                for page in file:
                        context += page.get_text()
        return context                      

In [383]:
def extrai_tabela(file,cpf,id):
    pg1,pg2 = tabula.read_pdf_with_template(f'{DATA_DIR}\\{file}',template_path=f'{BASE_DIR}\\tabula2.json')
    if pg1['Unnamed: 0'].isna().count() == len(pg1['Unnamed: 0']) and len(pg1.columns) == 13:
        del pg1['Unnamed: 0']
    colunas1 = ['MÊS', 'Unnamed: 0', 'Unnamed: 1', 'INSAL. %', 'VALOR', '6a PTE','FÉRIAS', 'Q. DIAS', 'B-01', 'V. INSAL.', '6aPTE', 'FÉRIAS.1','A RECEB.']
    colunas2 = ['MÊS', 'Unnamed: 1', 'INSAL. %', 'VALOR', '6a PTE','FÉRIAS', 'Q. DIAS', 'B-01', 'V. INSAL.', '6aPTE', 'FÉRIAS.1','A RECEB.']
    pg2.columns = colunas2
    pg1.set_index('MÊS', inplace = True)
    pg2.set_index('MÊS', inplace = True)
    pg = pg1.append(pg2)
    pg['CPF'] = cpf
    pg['ID'] = id
    return pg

### Extrair informações do cabeçalho

In [384]:
def extrai_cabecalho(context,id,file):
    #print(context)
    info_data = context
    nome = info_data[:info_data.find("RF:")].replace('NOME:','').strip()
    info_data = info_data[info_data.find("RF:"):]
    #print(nome)
    rf = (';').join(info_data[:info_data.find("CPF:")].replace('RF:','').strip().split('/'))
    info_data = info_data[info_data.find("CPF:"):]
    #print(rf)
    cpf = info_data[:info_data.find("Data Nascto.:")].replace('CPF:','').replace(' ','').replace('.','').replace('-','').strip()
    info_data = info_data[info_data.find("Data Nascto.:"):]
    #print(cpf)
    nascimento = info_data[:info_data.find("Proc. Jud. S/nº")].replace('Data Nascto.:','').strip()
    info_data = info_data[info_data.find("Proc. Jud. S/nº"):]
    #print(nascimento)
    judicial = info_data[:info_data.find("P.SEI.")].replace('Proc. Jud. S/nº','').strip()
    info_data = info_data[info_data.find("P.SEI."):]
    #print(judicial)
    sei = info_data[:info_data.find("Demonstrativos dos valores")].replace('P.SEI.','').strip()
    info_data = info_data[info_data.find("Demonstrativos dos valores"):]
    #print(sei)
    obs2 = info_data[:info_data.find("Período:")].replace('\n',' ').strip()
    info_data = info_data[info_data.find("Período:"):]
    #print(obs2)
    periodo = (';').join(info_data[:info_data.find("Situação funcional")].replace('Período:',' ').replace(' ','').strip().split('a'))
    info_data = info_data[info_data.find("Situação funcional"):]
    #print(periodo)
    situacao = info_data[:info_data.find("Obs:")].replace('Situação funcional','').replace(':','').replace(' ','').strip()
    info_data = info_data[info_data.find("Obs:"):]
    #print(situacao)
    obs = info_data[:info_data.find("\n")].replace('Obs:','').replace('\n',' ')
    info_data = info_data[info_data.find("\n"):]
    info_tabela = info_data
    #print(obs)
    info_data = f'{id};{nome};{rf};{cpf};{nascimento};{judicial};{sei};{periodo};{situacao};{obs2};{obs}'.replace('\n',' ')
    info_data = f'{info_data};\n'
    #print(tabela)
    tabela = extrai_tabela(file,cpf,id)
    print(nome)
    return info_data, tabela


### Salva arquivos

In [385]:
def save_file(SAVE_DIR,CONTEXT,NAME_FILE,FORMAT='txt'):
        '''
                Cria um arquivo com o conteúdo e salva na pasta.
                        SAVE_DIR  : Diretório para salvar o conteúdo
                        CONTEXT   : Conteúdo a ser salvo
                        NAME_FILE : Nome do arquivo para ser salvo
                        FORMAT    : Formato do arquivo final
        '''

        with open(f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}","a",encoding='utf-8') as output_file:
                output_file.write(CONTEXT)
        
        return f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}"

### Arquivo final

In [386]:
i = 0
CABECALHO = 'ID;NOME;RF;VINCULO;CPF;DATA_NASCIMENTO;PROCESSO_JUDICIAL;PROCESSO_SEI;PERIODO_INICIAL;PERIODO_FINAL;SITUACAO_FUNCIONAL;DESCRICAO;OBS\n'
save_file(BASE_DIR,CABECALHO,"servidores","csv")
for file in DATA_FILES:
    i += 1
    CONTEUDO = ler_dados(file)
    CABECALHO,TABELA = extrai_cabecalho(CONTEUDO,i,file)
    save_file(BASE_DIR,CABECALHO,"servidores","csv")
    print(i)

C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\3355230442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg = pg1.append(pg2)



ADENILSON SOUZA COSTA
1


C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\3355230442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg = pg1.append(pg2)



ADEVAIR PEREIRA DA SILVA
2


C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\3355230442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg = pg1.append(pg2)



...

C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\3355230442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg = pg1.append(pg2)



WALMIR CARDOMINGO
142


C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\3355230442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg = pg1.append(pg2)



WILKER COSTA PAES
143
WILLIAN ALVARENGA DE SOUZA
144


C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\3355230442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg = pg1.append(pg2)



In [401]:
file_test = f'{DATA_DIR}\\{DATA_FILES[0]}'

In [402]:
import tabula
cpf = '12345678910'
id = '1'

In [403]:
colunas = ['MÊS', 'Unnamed: 0', 'Unnamed: 1', 'INSAL. %', 'VALOR', '6a PTE','FÉRIAS', 'Q. DIAS', 'B-01', 'V. INSAL.', '6aPTE', 'FÉRIAS.1','A RECEB.']
colunas2 = ['MÊS', 'Unnamed: 1', 'INSAL. %', 'VALOR', '6a PTE','FÉRIAS', 'Q. DIAS', 'B-01', 'V. INSAL.', '6aPTE', 'FÉRIAS.1','A RECEB.']

In [416]:
pg1,pg2 = tabula.read_pdf_with_template(file_test,template_path=f'{BASE_DIR}\\tabula2.json')
#pg1['CPF'] = cpf
#pg1['ID'] = id
#pg2['CPF'] = cpf
#pg2['ID'] = id

In [423]:
len(pg2)

26

In [424]:
pg2.append(pg2.columns))

C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\1460474564.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg2.append(pg2.columns.values)



TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [392]:
pg1.columns

Index(['MÊS', 'Unnamed: 0', 'Unnamed: 1', 'INSAL. %', 'VALOR', '6a PTE',
       'FÉRIAS', 'Q. DIAS', 'B-01', 'V. INSAL.', '6aPTE', 'FÉRIAS.1',
       'A RECEB.'],
      dtype='object')

In [393]:
pg1.set_index('MÊS', inplace = True)

In [394]:
pg1

,Unnamed: 0,Unnamed: 1,INSAL. %,VALOR,6a PTE,FÉRIAS,Q. DIAS,B-01,V. INSAL.,6aPTE,FÉRIAS.1,A RECEB.
MÊS,,,,,,,,,,,,
MÊS,NaN,NaN,INSAL. %,VALOR,6a PTE,FÉRIAS,Q. DIAS,B-01,V. INSAL.,6aPTE,FÉRIAS,A RECEB.
mai-12,NaN,0.0,40%,"46,54","0,00","0,00",NaN,"444,10","177,64","0,00","0,00","131,10"
jun-12,NaN,0.0,40%,"46,54","0,00","0,00",NaN,"444,10","177,64","0,00","0,00","131,10"
jul-12,NaN,0.0,40%,"46,54","0,00","0,00",NaN,"444,10","177,64","0,00","0,00","131,10"
ago-12,NaN,0.0,40%,"46,54","0,00","0,00",NaN,"444,10","177,64","0,00","0,00","131,10"
...,...,...,...,...,...,...,...,...,...,...,...,...
nov-16,NaN,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"
dez-16,NaN,1.0,40%,"47,02","7,84","15,67",30,"755,15","302,06","50,34","100,69","382,56"
13o sal/16,NaN,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"


In [395]:
print(pg1['Unnamed: 0'].isna().count())
len(pg1['Unnamed: 0'])

64


64

In [396]:
del pg1['Unnamed: 0']

In [397]:
len(pg1.columns)

11

In [400]:
pg2.append(pg2.columns)

C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\60473067.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg2.append(pg2.columns)



TypeError: cannot concatenate object of type '<class 'pandas.core.indexes.base.Index'>'; only Series and DataFrame objs are valid

In [353]:
pg2.head()

,mar-17,1,40%,"7,84","0,00",Unnamed: 0,"755,15","302,06","50,34","0,00.1","297,55",Unnamed: 1
0,abr-17,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"
1,mai-17,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"
2,jun-17,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"
3,jul-17,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"
4,ago-17,1.0,40%,"47,02","7,84","0,00",NaN,"755,15","302,06","50,34","0,00","297,55"


In [354]:
len(pg2)

26

In [355]:
pg2.columns = colunas2

In [356]:
len(pg2.columns)

12

In [357]:
pg2.set_index('MÊS', inplace = True)

In [358]:
pg2.columns == pg1.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [359]:
pg3 = pg1.append(pg2)

C:\Users\vinicius.gomes\AppData\Local\Temp\ipykernel_9740\1099412048.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pg3 = pg1.append(pg2)



In [368]:
pg3.tail(60)

,Unnamed: 1,INSAL. %,VALOR,6a PTE,FÉRIAS,Q. DIAS,B-01,V. INSAL.,6aPTE,FÉRIAS.1,A RECEB.
MÊS,,,,,,,,,,,
ago-14,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
set-14,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
out-14,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
nov-14,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
dez-14,0.0,40%,"47,01","0,00","15,67",30,"755,07","302,03","0,00","100,68","340,02"
13o sal/14,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
jan-15,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
fev-15,0.0,40%,"47,01","0,00","0,00",NaN,"755,07","302,03","0,00","0,00","255,02"
mar-15,1.0,40%,"47,01","7,84","0,00",NaN,"755,07","302,03","50,34","0,00","297,52"


In [327]:
pg3['CPF'] = cpf
pg3['ID'] = id

In [332]:
pg3.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,  0,  1,  2,  3,
             4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
            21, 22, 23, 24, 25],
           dtype='int64')

In [370]:
def extrai_tabela(file,cpf,id):
    pg1,pg2 = tabula.read_pdf_with_template(f'{DATA_DIR}\\{file}',template_path=f'{BASE_DIR}\\tabula2.json')
    if pg1['Unnamed: 0'].isna().count() == len(pg1['Unnamed: 0']) and len(pg1.columns) == 13:
        del pg1['Unnamed: 0']
    pg1.set_index('MÊS', inplace = True)
    pg2.set_index('MÊS', inplace = True)
    pg = pg1.append(pg2)
    pg['CPF'] = cpf
    pg['ID'] = id
    return pg